# DSCI 525 - Web and Cloud Computing

***Milestone 4:*** In this milestone, you will deploy the machine learning model you trained in milestone 3.

You might want to go over the sample project (sampleproject.ipynb) in the milestone folder and get it done before starting this milestone. 

> Tutorial 3, is also going to help you with this. Attend-it/watch-recording if you need help.

Milestone 4 checklist :

- [x] Use an EC2 instance. You can spin up a new EC2 instance.
- [x] Develop your API here in this notebook (Question 1).
- [x] Copy it to ```app.py``` file in EC2 instance (Question 2).
- [x] Run your API for other consumers and test it among your colleagues.
- [x] Summarize your journey.

In this milestone, you will do certain things that you learned. For example...
- Setup an EC2 instance.
- Login to the instance.
- Work with Linux and use some basic commands.
- Configure the security group associated with your EC2 instance so that it accepts your webserver requests from your laptop.
- Configure AWS CLI.

In some places, I explicitly mentioned these to remind you.

In [3]:
## Import all the packages that you need
from flask import Flask, request, jsonify
import joblib
import numpy as np

## 1. Develop your API

rubric={mechanics:45}

You probably got how to set up primary URL endpoints from the `sampleproject.ipynb` and have them process and return some data. Here we are going to create a new endpoint that accepts a POST request of the features required to run the machine learning model that you trained and saved in the last milestone (i.e., a user will post the predictions of the 25 climate model rainfall predictions, i.e., features,  needed to predict with your machine learning model). Your code should then process this data, use your model to make a prediction, and return that prediction to the user. To get you started with all this, I've given you a template that you should fill out to set up this functionality:

> ***NOTE:*** You won't be able to test the flask module (or the API you make here) unless you go through steps in ```2. Deploy your API```. However, please make sure that you develop all your functions and inputs properly here.

```python
from flask import Flask, request, jsonify
import joblib
import numpy as np
## Import any other packages that are needed
app = Flask(__name__)
# 1. Load your model here
model = joblib.load('./model.joblib')
# 2. Define a prediction function
def return_prediction(arr):
    # format input_data here so that you can pass it to model.predict()
    return model.predict(np.array(arr).reshape(1, -1))
# 3. Set up home page using basic html
@app.route("/")
def index():
    # feel free to customize this if you like
    return """
    <h1>Welcome to our rain prediction service</h1>
    To use this service, make a JSON post request to the /predict url with 25 climate model outputs.
    """
# 4. define a new route which will accept POST requests and return model predictions
@app.route('/predict', methods=['POST'])
def rainfall_prediction():
    content = request.json  # this extracts the JSON content we sent
    prediction = return_prediction(content['data'])
    results = {"Input": content['data'], "Prediction": prediction[0]}  # return whatever data you wish, it can be just the prediction
                     # or it can be the prediction plus the input data, it's up to you
    return jsonify(results)
```

## 2. Deploy your API

rubric={mechanics:40}

Once your API (the question above is done) we're ready to deploy it! For this, do the following:

1. Setup an EC2 instance. Make sure you add a rule in security group (of your EC2 instance) to accept `All TCP` connections from `Anywhere`. After that SSH into your EC2 instance.
2. Make a file `app.py` file in your instance and copy what you developed in Question 1 in there. 

    2.1 You can use the Linux editor using ```vi```. More details on vi Editor [here](https://www.guru99.com/the-vi-editor.html). Use your previous learnings, notes, mini videos, etc. You can copy code from your jupyter and paste it into `app.py`.
    
    2.2 (OPTIONAL WAY) Or else you can make a file in your laptop called app.py and copy it over to your EC2 instance using ```scp```. Eg: ```scp -r -i "ggeorgeAD.pem" ~/Desktop/app.py  ubuntu@ec2-xxx.ca-central-1.compute.amazonaws.com:~/```

3. Download your model from s3 to your EC2 instance. You want to configure your AWS CLI and S3 for this. Use your previous learnings, notes, mini videos, etc.
4. Install the dependencies of your API, like `flask`, `joblib`, `sklearn`, etc...

    4.1. (Additional help) you can install the required packages inside your terminal.

        - Install packages (there might be others): 
            pip install flask scikit-learn joblib

5. Now you're ready to start your service, go ahead and run `flask run --host=0.0.0.0 --port=8080`. This will make your service available at your EC2 instance's `Public IPv4 address` on port 8080. Please ensure that you run this from where ```app.py``` and ```model.joblib``` reside.
6. You can now access your service by typing your EC2 instances `public IPv4 address` append with `:8080` into a browser, so something like `http://Public IPv4 address:8080`. From step 4, you might notice that flask output saying "Running on http://XXXX:8080/ (Press CTRL+C to quit)", where XXXX is `Private IPv4 address`, and you want to replace it with the `Public IPv4 address`
7. You should use `curl` to send a post request to your service to make sure it's working as expected.
>EG: curl -X POST http://your_EC2_public_ip:8080/predict -d '{"data":[1,2,3,4,53,11,22,37,41,53,11,24,31,44,53,11,22,35,42,53,12,23,31,42,53]}' -H "Content-Type: application/json"

8. Now, what happens if you exit your connection with the EC2 instance? Can you still reach your service? You won't be able to, because the `flask` process will shut down when you exit your shell session. This is because `flask` is running in the foreground, and when you exit your shell session, the process is killed. We need to run `flask` in the background so that it persists even after we exit our shell session.
9. We could use several options to help us persist our server even after we exit our shell session. We'll be using `screen`. `screen` will allow us to create a separate session within which we can run `flask` and won't shut down when we exit the main shell session. Read [this](https://linuxize.com/post/how-to-use-linux-screen/) to learn more on ```screen```.
10. Now, create a new `screen` session (think of this as a new, separate shell), using: `screen -S myapi`. If you want to list already created sessions do ```screen -list```. If you want to get into an existing ```screen -x myapi```.
11. Within that session, start up your flask app. You can then exit the session by pressing `Ctrl + A then press D`. Here you are detaching the session, once you log back into EC2 instance you can attach it using ```screen -x myapi```.
12. Feel free to exit your connection with the EC2 instance now and try reaccessing your service with `curl`. You should find that the service has now persisted!
13. ***CONGRATULATIONS!!!*** You have successfully got to the end of our milestones. Move to Task 3 and submit it.

#### Screenshot of success 
https://github.com/UBC-MDS/525-group-6/blob/main/milestone4/screenshots/success.png

![evidence_of_success](screenshots/success.png)

## 3. Summarize your journey from Milestone 1 to Milestone 4
rubric={reasoning:5, quality:5}
> Discuss with your team. While summarizing, you want to cover some key learnings from all milestones. There is no format or structure on how you write this. (also, no minimum number of words, but a soft max limit of 300 words).

In milestone 1, we downloaded data from figshare and combined multiple datasets into a single CSV file. We then performed exploratory data analysis (EDA) using Python and R. From this milestone, we learnt that partitioning data as a parquet file enables us to store and process our data more efficiently, and that using Apache arrow can significantly accelerate the process of transferring dataframes between Python and R. 

In milestone 2, we established an Amazon EC2 instance with JupyterHub and an S3 bucket. We transferred the data from milestone 1 to our S3 bucket and conducted further data wrangling to prepare for machine learning. From this milestone, we discovered that leveraging EC2 instances provides the scalability to simply adjust our computing capacity as needed, while S3 allows us to store data as objects in buckets with high storage capacity and excellent durability.

For milestone 3, we developed a RandomForestRegressor model and evaluated its performance against other individual climate models by comparing their root mean square error (RMSE). The ensemble model, RandomForestRegressor, demonstrated better performance with a lower RMSE, and we proceeded to fine-tune its hyperparameters. To accomplish this, we set up an EMR cluster and utilized a PySpark notebook to perform hyperparameter tuning.

After conducting hyperparameter tuning, we discovered that n_estimators = 100 and max_depth = 5 were the best-performing hyperparameters. Using these parameters, we trained our model and achieved a train RMSE of 8.10 and a test RMSE of 7.81. Our tuned model demonstrated reasonable performance, prompting us to save it for deployment in milestone 4. From this milestone, we learnt that EMR clusters provide us with a simple and efficient means for big data processing since they are scalable and exhibit high performance. Namely, the Spark framework enables efficient processing and analysis of large-scale data, such as training and testing machine learning models on large datasets.

During milestone 4, we deployed the machine learning model we trained in milestone 3 to build our API. Our deployment process involved creating an app.py file in our EC2 instance and installing all the necessary dependencies. Following this, we started a service session with Flask and successfully executed a post request to our service session. By providing input data, we were able to obtain the predicted amount of rainfall. From this milestone, we learnt that we can use Flask for our API development since it is a simple framework that is easy to set up and it integrates well with Python.


## 4. Submission instructions
rubric={mechanics:5}

In the textbox provided on Canvas please put a link to GitHub where TAs can find the following-
- [ ] This notebook with solution to Questions ```1 & 3```
- [ ] Screenshot from 
    - [ ] Output after trying curl. Following is a sample (`curl_deploy_sample.png` image in folder). This is just an example; your input/output doesn't have to look like this, you can design the way you like. But at a minimum, it should show your prediction value.

<image src="curl_deploy_sample.png" />